In [28]:
import time
import threading
# import multiprocessing
import multiprocess
import itertools
import os
import logging
import random
import string
import requests
from functools import partial
# from multiprocessing import Queue
from multiprocessing.pool import ThreadPool

# Definitions

In [29]:
# Processes are OS-level resource used to run and manage applications
# Processes have pid, list of open files and sockets, isolated memory,

In [30]:
# Special part of OS (scheduler) iterates over processes and give them time slot to run

# 2900 МГц
TIME_SLOT = 0.5/(1000*1000)
while True:
    for process in processes:
        process.run_for(TIME_SLOT)

NameError: name 'processes' is not defined

In [31]:
# Process has 1 or more threads
# Threads have common memory

In [32]:
# 2 kinds of tasks:
- parallel execution: speedup
- background execution: 
    - daemon process: OS services
    - multithreaded process: notifications, monitor, watcher, etc

SyntaxError: invalid syntax (Temp/ipykernel_21192/3031161495.py, line 2)

In [33]:
Parallel execution can be done in 2 ways:
    - threads
    - processes
    
Which one is better?

SyntaxError: invalid syntax (Temp/ipykernel_21192/3321894035.py, line 1)

# Threads

In [35]:
class timer():
    def __init__(self, message):
        self.message = message

    def __enter__(self):
        self.start = time.time()
        return None

    def __exit__(self, type, value, traceback):
        elapsed_time = (time.time() - self.start)
        print(self.message.format(elapsed_time))

TIME_TO_SLEEP = 4
        
def long_running_task(n=TIME_TO_SLEEP):
    print(threading.current_thread())
#     with timer('Elapsed 2: {}s'):
    time.sleep(n)

# what is it?
with timer('Executed in: {}s'):
#     long_running_task()
    long_running_task()


<_MainThread(MainThread, started 21332)>
Executed in: 4.010136604309082s


In [36]:
with timer('Executed in: {}s'):
    t1 = threading.Thread(target=long_running_task, args=(TIME_TO_SLEEP/2,))
    t2 = threading.Thread(target=long_running_task, args=(TIME_TO_SLEEP/2,))
    t1.start()
    t2.start()
    print(threading.current_thread())
    print('BEFORE T1 JOIN')
    t1.join() # be carefull
    print('BEFORE T2 JOIN')
    t2.join() 
    
print('next step')
# work with date

<Thread(Thread-141 (long_running_task), started 32348)>
<Thread(Thread-142 (long_running_task), started 28224)>
<_MainThread(MainThread, started 21332)>
BEFORE T1 JOIN
BEFORE T2 JOIN
Executed in: 2.0176119804382324s
next step


In [37]:
def run_threads(func, data, workers):
    threads = [
        threading.Thread(target=func, args=(data / workers, ))
        for _ in range(workers)
    ]
    for t in threads:
        t.start()
    for t in threads:
        t.join()

In [38]:
workers = 10
DATA_SIZE = 10

with timer('Elapsed: {}s'):
    run_threads(long_running_task, DATA_SIZE, workers)

<Thread(Thread-143 (long_running_task), started 29080)>
<Thread(Thread-144 (long_running_task), started 25688)><Thread(Thread-145 (long_running_task), started 13976)>

<Thread(Thread-146 (long_running_task), started 4380)>
<Thread(Thread-147 (long_running_task), started 30932)>
<Thread(Thread-148 (long_running_task), started 32632)>
<Thread(Thread-149 (long_running_task), started 22884)>
<Thread(Thread-150 (long_running_task), started 6412)>
<Thread(Thread-151 (long_running_task), started 32628)>
<Thread(Thread-152 (long_running_task), started 32424)>
Elapsed: 1.022937297821045s


In [ ]:
def generate_task_queue(total_tasks, queue_size):
    task_queue = []
    remainder = total_tasks
    while remainder > 0:
        value = min(remainder, random.randint(1, total_tasks//queue_size))
        remainder -= value
        task_queue.append(value)
    return task_queue

assert sum(generate_task_queue(1000, 20)) == 1000
assert sum(generate_task_queue(1000, 1)) == 1000
assert sum(generate_task_queue(1000, 1000)) == 1000
assert sum(generate_task_queue(1000, 42)) == 1000

In [39]:
# input_data = [DATA_SIZE / workers for _ in range(workers)]
input_data = [1, 2, 2, 1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
print(input_data)

with timer('Elapsed: {}s'):
    with ThreadPool(workers) as pool:
        pool.map(long_running_task, input_data)
#         for chunk in input_data:
#             long_running_task(chunk)

[1, 2, 2, 1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
<DummyProcess(Thread-153 (worker), started daemon 12284)>
<DummyProcess(Thread-154 (worker), started daemon 25628)>
<DummyProcess(Thread-155 (worker), started daemon 25452)>
<DummyProcess(Thread-156 (worker), started daemon 15692)>
<DummyProcess(Thread-157 (worker), started daemon 14912)>
<DummyProcess(Thread-158 (worker), started daemon 25892)>
<DummyProcess(Thread-159 (worker), started daemon 2932)>
<DummyProcess(Thread-160 (worker), started daemon 15836)>
<DummyProcess(Thread-161 (worker), started daemon 25144)>
<DummyProcess(Thread-162 (worker), started daemon 32004)>
Elapsed: 2.0242984294891357s


<img src="https://www.nginx.com/wp-content/uploads/2016/07/thread-pools-worker-process-event-cycle.png">

# Real world task

In [40]:
def fetch_pic(num_pic):
# def fetch_pic(num_pic, path):
    url = 'https://picsum.photos/400/600'
    path = os.path.join(os.getcwd(), 'img')
    for _ in range(num_pic):
        random_name = ''.join(random.choices(string.ascii_letters + string.digits, k=5))
        response = requests.get(url)
        if response.status_code == 200:
            with open(f'{path}/{random_name}.jpg', 'wb') as f:
                f.write(response.content)
                #print(f"Fetched pic [{os.getpid()}]: {f.name}")

In [50]:
workers = 256
DATA_SIZE = 100

with timer('Elapsed: {}s'):
#     with multiprocessing.Pool(workers) as pool:
    with ThreadPool(workers) as pool:
        input_data = [DATA_SIZE // workers for _ in range(workers)]
        # input_data = [(DATA_SIZE // workers, './pics') for _ in range(workers)]
        pool.map(fetch_pic, input_data)
# 40.52 - 1 
# 19.54 - 2 
# 10.93 - 4
# 4.57  - 8 
# 2.42 - 16
# 1.43 - 32
# 0.77 - 64
# 0.02 - 128
# 0.06 - 256

Elapsed: 0.06856846809387207s


# IO vs CPU bound tasks

In [51]:
DATA_SIZE = 1_000_000
lst2 = []
lst1 = []


def fill_data2(n):
    # print(threading.current_thread())
    while n > 0:
        n -= 1
        lst2.append(random.randint(1, 100))

def fill_data1(n):
    # print(threading.current_thread())
    while n > 0:
        n -= 1
        lst1.append(random.randint(1, 100))        
        
with timer('Elapsed: {}s'):
#     fill_data(DATA_SIZE, lst)
    fill_data1(DATA_SIZE)

Elapsed: 0.5714006423950195s


In [ ]:
a=0 
 1     2  M
 |     | 
 a=1  a=0
      a=2
       X
 

Global Interpreter Lock
1  2  M
   |  |
|     |
   |  |
|     |
   |  |
______
GIL

In [52]:
with timer('Elapsed: {}s'):
#     t1 = threading.Thread(target=fill_data, args=(DATA_SIZE // 2, lst))
#     t2 = threading.Thread(target=fill_data, args=(DATA_SIZE // 2, lst))
    t1 = threading.Thread(target=fill_data2, args=(DATA_SIZE // 2,))
    t2 = threading.Thread(target=fill_data1, args=(DATA_SIZE // 2,))
    t1.start()
    t2.start()
    t1.join()
    t2.join()
    lst = lst2 + lst1
    
print(len(lst), lst[:100])    

Elapsed: 0.5763282775878906s
2000000 [38, 76, 9, 29, 78, 21, 61, 10, 90, 80, 2, 26, 58, 54, 46, 95, 53, 48, 76, 34, 22, 12, 79, 53, 78, 38, 86, 38, 32, 74, 39, 35, 52, 60, 11, 60, 71, 13, 60, 63, 59, 20, 33, 84, 71, 59, 2, 87, 20, 52, 67, 17, 96, 83, 93, 10, 19, 46, 56, 9, 45, 37, 74, 8, 10, 93, 2, 99, 62, 97, 15, 96, 53, 49, 72, 20, 94, 16, 15, 56, 47, 97, 34, 16, 84, 83, 30, 36, 1, 83, 98, 71, 88, 1, 9, 5, 76, 96, 56, 68]


In [53]:
lst = []
workers = 8
with timer('Elapsed: {}s'):
    with ThreadPool(workers) as pool:
        input_data = [DATA_SIZE // workers for _ in range(workers)]
#         pool.map(partial(fill_data, lst=lst), input_data)
        result = pool.map(fill_data1, input_data)
        
print(len(lst), lst[:100])

Elapsed: 0.58380126953125s
0 []


In [62]:
lst = []
workers = 8

DATA_SIZE = 1_000_000
lst = []


def fill_data(n):
    # print(threading.current_thread())
    import random
    lst = []
    while n > 0:
        n -= 1
        lst.append(random.randint(1, 100))
        
with timer('Elapsed: {}s'):
    with multiprocess.Pool(workers) as pool:
        input_data = [DATA_SIZE // workers for _ in range(workers)]
        pool.map(fill_data, input_data)
        
print(len(lst), lst[:100])

# 0.70 - 1
# 0.41 - 2
# 0.31 - 4
# 0.34 - 8

Elapsed: 0.40836119651794434s
0 []


In [59]:
def factorize_naive(n):
    """ A naive factorization method. Take integer 'n', return list of
        factors.
    """
    if n < 2:
        return []
    factors = []
    p = 2

    while True:
        if n == 1:
            return factors

        r = n % p
        if r == 0:
            factors.append(p)
            n = n / p
        elif p * p >= n:
            factors.append(n)
            return factors
        elif p > 2:
            # Advance in steps of 2 over odd numbers
            p += 2
        else:
            # If p == 2, get to 3
            p += 1

    assert False, "unreachable"

In [ ]:
DATA_SIZE = 1_000

In [ ]:
result = {}
workers = 16
with timer('Elapsed: {}s'):
    with multiprocess.Pool(workers) as pool:
        input_data = (i for i in range(1, DATA_SIZE+1))
        result = [
            (n, factors)
            for n, factors in enumerate(pool.map(factorize_naive, input_data), 1)
        ]
    
print(len(result), result[:100])

In [ ]:
result = {}
workers = 16
with timer('Elapsed: {}s'):
    with ThreadPool(workers) as pool:
        input_data = (i for i in range(1, DATA_SIZE+1))
        result = [
            (n, factors)
            for n, factors in enumerate(pool.map(factorize_naive, input_data), 1)
        ]
    
print(len(result), result[:100])